In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from config import weather_api_key
import time
from datetime import datetime

In [2]:
lats=np.random.uniform(-90, 90, size=1500)
lngs=np.random.uniform(-180,180,size=1500)
lat_lng=zip(lats,lngs)
lat_lng

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lng)
coordinates

[(-26.966972420378198, 151.38658017415742),
 (62.60398576748619, 80.99031669459578),
 (-79.852295681772, -109.35215897442447),
 (13.548005220270056, -179.11312625997158),
 (66.393601523916, -19.590951653586586),
 (17.552683051783376, 161.06362269740322),
 (51.051442522953266, 57.43837021200105),
 (-53.75370887873032, 82.33391328778737),
 (11.641518114024905, 105.47845433403518),
 (-62.98804737988266, -33.20471743079372),
 (16.731341835688113, 5.399758057656641),
 (-1.1965371832478127, -24.654734624247908),
 (28.65597511959531, -14.117934906773996),
 (-19.158517233444996, -35.96581148601456),
 (52.40785925094053, -20.27864156082282),
 (-11.171249478789207, 55.92805726598428),
 (-45.76213537953262, -69.8120837795256),
 (-15.422032457855153, 167.43134063191962),
 (69.02712333620127, 102.72978572350053),
 (67.07744757740855, 119.58542708971811),
 (-67.10023904281704, 29.05788432176533),
 (24.773872833675284, -126.91080071480438),
 (73.43834262234677, -166.59978622358554),
 (-84.85704579312

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
print(len(cities))
print(cities[1])

631
krasnoselkup


In [5]:
url= "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=910799ed5338378d0da7cc7a5308ebb1


### Why? Unfortunately, this will cause programming errors when we are building the city_url because it adds the index, not the city name, to the city_url.

### why city.title()

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

for i, city in enumerate(cities):
    if (i%50 == 0 and i >= 50):
        set_count += 1
        record_count=1
    
   
    city_url = url + "&q=" + city.replace(" ", "+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    
    # Run an API request for each of the cities.
    try:
        city_weather= requests.get(city_url).json()
        
        city_lat= city_weather['coord']['lat']
        city_lng= city_weather['coord']['lng']
        city_max_temp= city_weather['main']['temp_max']
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list.
        city.data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
            
        # If an error is experienced, skip the city.
    except Exception as e:
        print("City not found. Skipping...")
        pass
        
    #except Exception as e:     
        #print(e)
        

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
        

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | dalby
City not found. Skipping...
Processing Record 2 of Set 1 | krasnoselkup
City not found. Skipping...
Processing Record 3 of Set 1 | punta arenas
City not found. Skipping...
Processing Record 4 of Set 1 | kapaa
City not found. Skipping...
Processing Record 5 of Set 1 | skagastrond
City not found. Skipping...
Processing Record 6 of Set 1 | butaritari
City not found. Skipping...
Processing Record 7 of Set 1 | mednogorsk
City not found. Skipping...
Processing Record 8 of Set 1 | busselton
City not found. Skipping...
Processing Record 9 of Set 1 | kampong cham
City not found. Skipping...
Processing Record 10 of Set 1 | mar del plata
City not found. Skipping...
Processing Record 11 of Set 1 | abalak
City not found. Skipping...
Processing Record 12 of Set 1 | natal
City not found. Skipping...
Processing Record 13 of Set 1 | puerto del rosario
City not found. Skipping...
Processing Record 14 of Set 

KeyboardInterrupt: 

In [8]:
city_data_df= pd.DataFrame(city_data)
city_data_df.head()

""


In [ ]:
new_col_order= ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudines", "Wind Speed"]
city_data_df= city_data_df[new_col_order]


In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")
